In [1]:
import pandas as pd
import numpy as np
import io
import os
import json
import distutils.dir_util
from collections import Counter
import tensorflow as tf
from tensorflow.keras.models import save_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense, SimpleRNN,Dropout

## 데이터 전처리

In [ ]:
# from google.colab import drive
# drive.mount('/gdrive')

In [2]:
# train data 준비
f_train = pd.read_json('../kakao_arena_melon/arena_data/orig/train.json' ,typ = 'frame', encoding='utf-8')
df_train = pd.DataFrame(f_train)
df_train = df_train.sort_values(by=['like_cnt'],ascending=False)
df_train = df_train[df_train['like_cnt']>5]
df_train.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
8699,"[노동요, 취향저격, 드라이브, Pop, 기분전환, 트렌디, 스트레스, 신나는]",74220,HOT TRENDY POP: 놓쳐선 안될 'POP' (매주 업데이트),"[159317, 368264, 600999, 274905, 643023, 24070...",53211,2020-04-17 14:32:23.000
61103,"[감성, 새벽, 때껄룩, 잔잔한, 휴식, 팝송, 떼껄룩, 힐링, 팝, 신나는]",122388,❤️때껄룩님 ᴛᴀᴋᴇ ᴀ ʟᴏᴏᴋ 플레이리스트❤️,"[532771, 383775, 432753, 176384, 152694, 60126...",41844,2020-04-23 20:58:45.000
19863,"[매장음악, 카페, 러블리쏘, 분위기, 드라이브, 여행, 매력적인, 기분전환, 팝]",26174,듣다보면 '우와!' 하고 제목을 보게되는 팝,"[40160, 20039, 561488, 20051, 16242, 256568, 7...",27268,2020-04-19 21:22:11.000


In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53186 entries, 8699 to 31216
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          53186 non-null  object
 1   id            53186 non-null  int64 
 2   plylst_title  53186 non-null  object
 3   songs         53186 non-null  object
 4   like_cnt      53186 non-null  int64 
 5   updt_date     53186 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.8+ MB


In [4]:
# test data 준비
f_tst = pd.read_json('../kakao_arena_melon/arena_data/questions/val.json' ,typ = 'frame', encoding="utf-8")
df_tst = pd.DataFrame(f_tst)
df_tst = df_tst.sort_values(by=['like_cnt'],ascending=False)
# df_tst = df_tst[df_tst['like_cnt']>5]
df_tst.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
6561,"[힐링, 설렘, 사랑]",102123,여자들 취향저격하는 달달한 노래,"[481910, 460884, 547506, 106365, 288472, 20118...",20149,2019-11-08 05:31:14.000
11929,"[밤, 새벽, 밤에듣기좋은노래, 새벽에듣기좋은노래, 공부]",11762,불 꺼진 방 침대 속 자기 직전... 듣기 좋은 노래 ⭐,[],19758,2020-04-17 03:16:32.000
19718,[],56212,내가 뽑은 최고의 팝송 TOP 100,"[430846, 19486, 428272, 284554, 319715, 669677...",18403,2012-03-19 10:49:17.000
7530,"[까페, 편안한]",49159,편하게 듣는 사랑받는 팝 BEST,[],16605,2016-08-15 18:31:56.000
20488,"[연주곡, 피아노, 수험생, 뉴에이지]",46039,공부할때 마다 듣는 잔잔한 피아노,[],14371,2018-05-04 21:18:04.000


In [5]:
df_tst.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23015 entries, 6561 to 13603
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          23015 non-null  object
 1   id            23015 non-null  int64 
 2   plylst_title  23015 non-null  object
 3   songs         23015 non-null  object
 4   like_cnt      23015 non-null  int64 
 5   updt_date     23015 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.2+ MB


In [6]:
# 플레이리스트 내 song id 리스트
train_song = df_train['songs']
# 플레이리스트 내 tag 리스트
train_tag = df_train['tags']
# 플레이리스트 내 song id 리스트 전체 나열
song_list = [song for plist in train_song for song in plist]
# 플레이리스트 내 tag 리스트 전체 나열
tag_list = [tag for plist in train_tag for tag in plist]

In [7]:
# 전체 나열 리스트 중 각 song의 개수
count_song = Counter(song_list)
# 전체 나열 리스트 중 각 tag의 개수
count_tag = Counter(tag_list)

In [8]:
# 전체 플레이리스트 내 20회 넘게 담긴 song을 dictionary에 추가
mfl={}  # meaningful
for key, value in count_song.items():
    if value>20:
        mfl[key]=value
song_len = len(mfl) # 20회 이상 담긴 song의 개수
song_len

24537

In [9]:
# 전체 플레이리스트 내 10회 넘게 담긴 tag을 dictionary에 추가
for key, value in count_tag.items():
    if value>10:
        mfl[key]=value
tag_len=len(mfl)-song_len # 11회 이상 담긴 tag의 개수
tag_len

1619

In [10]:
mfl_col = list(mfl.keys())
len(mfl_col) # 유의미한 song과 tag의 총 개수 (song_len+tag_len)

26156

In [13]:
# 전체 플레이리스트를 row, 유의미한 song과 tag를 column으로 하는 zero matrix 생성
zero_matrix = np.zeros((len(df_train),len(mfl_col)))
zero_matrix.shape

(53186, 26156)

In [14]:
df_zero = pd.DataFrame(zero_matrix,columns=mfl_col,index=df_train['id'])
df_zero.head() # matrix >> dataframe 변환

,643023,452196,424328,380410,248200,621033,443160,532771,383775,432753,176384,152694,330285,142570,12397,559731,477991,515378,312626,495884,687008,295361,433444,258083,368753,225303,309245,26977,467970,185211,583390,439775,522895,592295,629725,141762,639765,501076,614472,6107,...,쌀쌀,고막힐링,1950_60,국내랩힙합,로멘틱,바이브,B,코노,입학,띵곡만,128BPM,휠링,나만의Best3,피쉬슈즈,신난_흥겨운,트로이시반,에이핑크,살빼자,아이오아이,모타운,걷기,넬,레어한,졸음방지,러블리즈,빌보드핫100,필청곡,빌보드핫차트,LilNasX,시대별차트,국내차트,끝,청하,너도알았으면,산뜻,찾아오는DJ,스쿨오브록,2월,푸른밤종현입니다,힙합리스트
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
74220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26174,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72658,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# 각 플레이리스트에 해당하는 곡, 태그 좌표에 1 부여

def create_onehot(df,column_name):
    zero_matrix=np.zeros((len(df),len(column_name)))
    zero_df=pd.DataFrame(zero_matrix,columns=column_name,index=df['id'])
    for i in range(len(df)):
        for j in df.iloc[i,0]+df.iloc[i,3] :
            if j in column_name:
                zero_df.iloc[i,column_name.index(j)]=1
    return zero_df

In [38]:
# create_onehot 함수 활용 train data >> onehot encoding
train_onehot = create_onehot(df_train, mfl_col)
train_onehot.shape

(53186, 26156)

In [12]:
# create_onehot 함수 활용 validation data >> onehot encoding
tst_onehot = create_onehot(df_tst, mfl_col)
tst_onehot.shape

(23015, 26156)

## 오토인코더(학습)

In [41]:
# encoding_dim=64
# input_pl = Input(shape=(len(mfl_col),)) # one-hot encoding된 DataFrame의 행이 Input
# dropout = Dropout(0.2)(input_pl)
# encoded = Dense(encoding_dim,activation='relu')(input_pl) # 64개로 축소(인코딩)
# encoded = Dense(36,activation='relu')(encoded) # 36개로 축소(인코딩)
# decoded = Dense(len(mfl_col),activation='sigmoid')(encoded) # 다시 원래 column 개수로 복원(디코딩)
# autoencoder = Model(input_pl,decoded)
# autoencoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 26156)]           0         
                                                                 
 dense_3 (Dense)             (None, 64)                1674048   
                                                                 
 dense_4 (Dense)             (None, 36)                2340      
                                                                 
 dense_5 (Dense)             (None, 26156)             967772    
                                                                 
Total params: 2644160 (10.09 MB)
Trainable params: 2644160 (10.09 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
# autoencoder.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [43]:
# history = autoencoder.fit(train_onehot,train_onehot,epochs=10,batch_size=64,validation_split=0.2,verbose=1) # 기존의 데이터프레임 복원이 목적이기 때문에, test data == train data

Epoch 1/10
665/665 [==============================] - 374s 560ms/step - loss: 0.0592 - accuracy: 0.0035 - val_loss: 0.0093 - val_accuracy: 0.0026
Epoch 2/10
665/665 [==============================] - 333s 500ms/step - loss: 0.0103 - accuracy: 0.0037 - val_loss: 0.0082 - val_accuracy: 0.0026
Epoch 3/10
665/665 [==============================] - 364s 547ms/step - loss: 0.0087 - accuracy: 0.0040 - val_loss: 0.0071 - val_accuracy: 0.0027
Epoch 4/10
665/665 [==============================] - 373s 560ms/step - loss: 0.0077 - accuracy: 0.0082 - val_loss: 0.0065 - val_accuracy: 0.0177
Epoch 5/10
665/665 [==============================] - 357s 536ms/step - loss: 0.0071 - accuracy: 0.0200 - val_loss: 0.0062 - val_accuracy: 0.0187
Epoch 6/10
665/665 [==============================] - 362s 545ms/step - loss: 0.0067 - accuracy: 0.0262 - val_loss: 0.0059 - val_accuracy: 0.0234
Epoch 7/10
665/665 [==============================] - 365s 548ms/step - loss: 0.0064 - accuracy: 0.0297 - val_loss: 0.0057 -

In [44]:
# autoencoder.save('autoencoder_basic.h5')

C:\Users\Playdata\AppData\Roaming\Python\Python38\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
autoencoder = tf.keras.models.load_model('autoencoder_basic_ad.h5')

In [14]:
predict_plist = autoencoder.predict(tst_onehot)

720/720 [==============================] - 7s 9ms/step


In [15]:
df_id = list(df_tst['id'])
col= mfl_col
ori_song = col[:song_len]
ori_tag = col[song_len:]

song_predict = predict_plist[:,:song_len] # song output(추천곡)
tag_predict = predict_plist[:,song_len:] # tag output(추천태그)

In [16]:
result=[]
n=0
for i in df_id:
    dic={}
    dic['id']=i

    plist_song=song_predict[n].argsort()[-100:] # predict한 song output 중 상위 100개
    p_song=[]
    for song in plist_song:
        p_song.append(ori_song[song])
    dic['songs']=p_song

    plist_tag=tag_predict[n].argsort()[-10:] # predict한 tag output 중 상위 10개
    p_tag=[]
    for tag in plist_tag:
        p_tag.append(ori_tag[tag])
    dic['tags']=p_tag
    n+=1
    result.append(dic)

In [17]:
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname+".json")
    distutils.dir_util.mkpath(parent)
    with io.open( fname+".json", "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [18]:
write_json(result,'results2_ad')